In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, classification_report
from xgboost import XGBClassifier

In [2]:

# Load the dataset
df = pd.read_csv("./queryResults/samples.csv")

In [3]:
# Clean column names (standardize to lowercase and remove prefixes like 'samples.')
df.columns = [col.split('.')[-1].strip().lower() for col in df.columns]
print("✅ Columns in dataset:", df.columns.tolist())

# Confirm target column
if 'delayed' in df.columns:
    target_column = 'delayed'
else:
    raise ValueError("❌ 'delayed' column not found in dataset.")

✅ Columns in dataset: ['dayofmonth', 'dayofweek', 'deptime', 'crsdeptime', 'uniquecarrier', 'flightnum', 'tailnum', 'actualelapsedtime', 'crselapsedtime', 'origin', 'dest', 'distance', 'delayed']


In [4]:
# Drop missing values
df.dropna(inplace=True)

# Drop non-useful or non-numeric columns
columns_to_drop = ['tailnum', 'flightnum']  # You can add more if needed
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

# Encode categorical variables
categorical_cols = ['uniquecarrier', 'origin', 'dest']
df = pd.get_dummies(df, columns=[col for col in categorical_cols if col in df.columns], drop_first=True)

In [5]:
# Prepare X and y
X = df.drop(columns=[target_column])
y = df[target_column].map({'N': 0, 'Y': 1}) if df[target_column].dtype == object else df[target_column]


In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Train XGBoost model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [10]:
# Predict and evaluate
y_pred = model.predict(X_test)

In [11]:
# Evaluate
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print("🎯 Accuracy:", round(accuracy * 100, 2), "%")
print("🎯 Precision:", round(precision * 100, 2), "%")
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))


🎯 Accuracy: 93.25 %
🎯 Precision: 96.42 %

📊 Classification Report:
               precision    recall  f1-score   support

       False       0.90      0.97      0.93     19966
        True       0.96      0.90      0.93     20034

    accuracy                           0.93     40000
   macro avg       0.93      0.93      0.93     40000
weighted avg       0.93      0.93      0.93     40000

